In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
from scipy import stats
from colour import Color
import random

In [2]:
## Dicts

brands_dict = {1: 'Wilfred', 2: 'Wilfred Free', 3: 'Babaton', 4: 'The Group by Babaton', 
              5: 'Babaton 1-01', 6: 'Ten by Babaton', 7: 'Tna', 8: 'TnAction', 
              9: 'Super World', 10: 'Sundary Best', 11: 'Denim Forum', 12: 'Auxiliary', 
              13: 'Talula'}

aesthetic_dict = {'Wilfred': 'Frenchy', 'Wilfred Free': 'Casual', 'Babaton': 'Opulent', 'The Group by Babaton': 'Soft', 
              'Babaton 1-01': 'Academia', 'Ten by Babaton': 'Party', 'Tna': 'Athluxury' , 'TnAction': 'Sporty', 
              'Super World': 'Cozy', 'Sundary Best': 'Youthful', 'Denim Forum': 'Picturesque', 'Auxiliary': 'Mix & Match', 
              'Talula': 'Wild Child'}

month_dict =  {'January': 'Bohemian', 'February': 'Romance', 'March': 'Spring', 'April': 'Floral', 
               'May': 'Chic', 'June': 'Neon', 'July': 'Tropics', 'August': 'Glitz', 
               'September': 'Smart', 'October': 'Glam', 'November': 'Comfort', 'December': 'Wonderland'}


In [3]:
## Define functions used

def check_color(color):
    try:
        Color(color)
        if color == '':
            return False
        return True
    except ValueError:
        return False
    
def replace_colour(description):
    c = [i for i in description.split(' ') if check_color(i)]
    if len(c) == 0:
        return 'COLOURLESS'
    else:
        return c[0]

In [4]:
## Metadata

# Load & clean data
retail_data = pd.read_excel('online_retail_II.xlsx')



retail_data.rename(columns={'InvoiceDate': 'Invoice_Date', 'Customer ID': 'Customer_ID'}, inplace=True)

# retail_data = retail_data[['Description', 'Quantity', 'Invoice_Date', 'Price', 'Customer_ID', 'Country']]
retail_data = retail_data.dropna(subset=['Description', 'Quantity', 'Invoice_Date', 'Price', 'Customer_ID', 'Country'])

# Add month field
retail_data['Month'] = retail_data.apply(lambda x: x['Invoice_Date'].month_name(), axis=1)
retail_data['Colour'] = retail_data.apply(lambda x: replace_colour(x['Description']), axis=1)
retail_data['Colour'] = retail_data.apply(lambda x: x['Colour'].title(), axis=1)



# Assign brands
random.seed(10)

brand_arr = np.random.randint(low=1, high=14, size=len(retail_data))
Brand = [brands_dict[num] for num in brand_arr]
retail_data['Brand'] = Brand

retail_data


,Invoice,StockCode,Description,Quantity,Invoice_Date,Price,Customer_ID,Country,Month,Colour,Brand
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,December,Colourless,The Group by Babaton
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,December,Pink,Tna
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,December,White,Denim Forum
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,December,Colourless,Sundary Best
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,December,Colourless,TnAction
...,...,...,...,...,...,...,...,...,...,...,...
525456,538171,22271,FELTCRAFT DOLL ROSIE,2,2010-12-09 20:01:00,2.95,17530.0,United Kingdom,December,Colourless,Sundary Best
525457,538171,22750,FELTCRAFT PRINCESS LOLA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,December,Colourless,The Group by Babaton
525458,538171,22751,FELTCRAFT PRINCESS OLIVIA DOLL,1,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,December,Colourless,Talula
525459,538171,20970,PINK FLORAL FELTCRAFT SHOULDER BAG,2,2010-12-09 20:01:00,3.75,17530.0,United Kingdom,December,Pink,Wilfred Free


In [5]:
retail_data.describe()

,Quantity,Price,Customer_ID
count,417534.000000,417534.000000,417534.000000
mean,12.758815,3.887547,15360.645478
std,101.220424,71.131797,1680.811316
min,-9360.000000,0.000000,12346.000000
25%,2.000000,1.250000,13983.000000
50%,4.000000,1.950000,15311.000000
75%,12.000000,3.750000,16799.000000
max,19152.000000,25111.090000,18287.000000


In [6]:
## Total items purchased df 

# Find all customers
cust_ids = retail_data['Customer_ID'].unique().tolist()


quantity_df = pd.DataFrame(columns = ['Customer_ID', 'Num_Items'])

for cust in cust_ids:
    orders = retail_data[retail_data['Customer_ID'] == cust]['Quantity']
    quantity_df.loc[str(cust_ids.index(cust))] = [cust, sum(orders)]
    
quantity_df

,Customer_ID,Num_Items
0,13085.0,727.0
1,13078.0,6466.0
2,15362.0,368.0
3,18102.0,122988.0
4,12682.0,5434.0
...,...,...
4378,18269.0,76.0
4379,13270.0,200.0
4380,12942.0,69.0
4381,13369.0,149.0


In [7]:
# Create empty aggregated df
agg_df = pd.DataFrame(columns = ['Customer_ID', 'Total_Spent', 'Num_Items', 'Num_Visits', 'Num_Items_Percentile', 'Top_Brand', 'Second_Brand', 'Third_Brand', 'Top_Month', 'Num_Items_Top_Month', 'Num_Visits_Top_Month', 'Top_Colour', 'Aesthetic', 'Individuality_Index', 'Twindex'])


for cust in cust_ids:
    # Subset df from customer purchases
    cust_df = retail_data[retail_data['Customer_ID'] == cust]
    
    cust_id = cust
    
    # Include returns in total spent
    tot_spent = sum(cust_df['Quantity']*cust_df['Price'])
    
    # Filter out returns
    cust_df = cust_df[cust_df['Quantity'] > 0]
    
    # Skip any customers with negative total purchases (returned more than they purchased)
    if not cust_df.empty:
        num_items = quantity_df[quantity_df['Customer_ID'] == cust]['Num_Items']
        
        num_visits = len(cust_df['Invoice_Date'].unique())
        
        num_items_percentile = stats.percentileofscore(quantity_df['Num_Items'], num_items)[0]
        
        
        top_three_brands = cust_df['Brand'].value_counts().nlargest(3).index.to_list()
        top_brand = top_three_brands[0]
        
        if len(top_three_brands) > 1:
            second_brand = top_three_brands[1]
        else: 
            second_brand = " "
        
        if len(top_three_brands) > 2:
            third_brand = top_three_brands[2]
        else: 
            third_brand = " "
        
        top_month = cust_df['Month'].mode()[0]
        num_items_top_month = sum(cust_df[cust_df['Month'] == top_month]['Quantity'])
        num_visits_top_month = len(cust_df[cust_df['Month'] == top_month]['Invoice_Date'].unique())
        
        
        top_colour = cust_df['Colour'].mode()[0]
        
        aesthetic = aesthetic_dict[top_brand] + ' ' + month_dict[top_month]
         
        ind_indx = 0                         
        twindex = 0
 
        agg_df.loc[str(cust_ids.index(cust))] = [cust_id, tot_spent, num_items[0], num_visits, num_items_percentile, top_brand, second_brand, third_brand, top_month, num_items_top_month, num_visits_top_month, top_colour, aesthetic, ind_indx, twindex]
 

In [8]:
def find_individuality(aes, aes_lst):
    twindex = len([x for x in aes_lst if x == aes]) - 1
    ind_indx = (twindex+1)/len(aes_lst)
    return ind_indx, twindex
    

agg_df['Individuality_Index'] = agg_df.apply(lambda x: find_individuality(x['Aesthetic'], agg_df['Aesthetic'].tolist())[0], axis=1)
agg_df['Twindex'] = agg_df.apply(lambda x: find_individuality(x['Aesthetic'], agg_df['Aesthetic'].tolist())[1], axis=1)


In [9]:
agg_df

# Customer_ID: Customer ID maintained over several purchases
# Total_Spent: Total $$ spent in recorded purchase history (accounting for returns)
# Num_Items: Total number of items purchases recorded (accounting for returns)
# Num_Items_Percentile: Percentile of number of items purchased
# Top_Brand: Mode of Brand purchased
# Second_Brand: Second most frequently purchased brand, empty string if none
# Third_Brand: Second most frequently purchased brand, empty string if none
# Top_Month: Mode of purchases made in each month
# Top_Colour: Mode of colour of items purchased
# Aesthetic: Overall aesthetic of customer based on Top_Brand and Top_Month
# Individuality_Index: Proportion of customers with Aesthetic
# Twindex: Number of other customers (not including current customer) with Aesthetic

,Customer_ID,Total_Spent,Num_Items,Num_Visits,Num_Items_Percentile,Top_Brand,Second_Brand,Third_Brand,Top_Month,Num_Items_Top_Month,Num_Visits_Top_Month,Top_Colour,Aesthetic,Individuality_Index,Twindex
0,13085.0,1187.08,727.0,6,67.989961,Sundary Best,Denim Forum,Talula,January,352,2,Colourless,Youthful Bohemian,0.003245,13
1,13078.0,16451.55,6466.0,32,98.037874,Auxiliary,Denim Forum,Sundary Best,November,1345,4,Colourless,Mix & Match Comfort,0.014372,61
2,15362.0,613.08,368.0,2,50.193931,Babaton 1-01,Ten by Babaton,Denim Forum,December,145,1,Colourless,Academia Wonderland,0.010431,44
3,18102.0,341776.73,122988.0,85,99.931554,Babaton,Auxiliary,Ten by Babaton,April,7797,10,Colourless,Opulent Floral,0.005100,21
4,12682.0,11657.59,5434.0,23,97.444673,Wilfred,Babaton,Denim Forum,December,891,5,Colourless,Frenchy Wonderland,0.008345,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4378,18269.0,168.60,76.0,1,12.776637,Babaton 1-01,Sundary Best,Denim Forum,December,76,1,Colourless,Academia Wonderland,0.010431,44
4379,13270.0,590.00,200.0,1,32.546201,Wilfred,,,December,200,1,Colourless,Frenchy Wonderland,0.008345,35
4380,12942.0,258.75,69.0,1,11.350673,Denim Forum,Babaton,Sundary Best,December,69,1,Colourless,Picturesque Wonderland,0.010431,44
4381,13369.0,308.28,149.0,1,25.655943,Sundary Best,Babaton 1-01,Wilfred Free,December,149,1,Colourless,Youthful Wonderland,0.009736,41


In [10]:
agg_df.to_json('customer_summary.json')